# Distances web crawler + API test


In [2]:
import pandas as pd;
import requests
import unittest
from bs4 import BeautifulSoup
#from multiprocessing import Pool
from ipywidgets import widgets
from IPython.display import display
#import json
import re


In [3]:
website_main = "https://fr.wikipedia.org/wiki/Liste_des_communes_de_France_les_plus_peupl%C3%A9es"
website_prefix = "https://www.distance24.org/route.json?stops="


In [4]:
def _handle_request_result_and_build_soup(request_result):
    if request_result.status_code == 200:
        html_doc =  request_result.text
        soup = BeautifulSoup(html_doc,"html.parser")
        return soup


In [5]:
def _convert_string_to_float(string):
    temp = string.strip()
    if temp[0]== '(' : # small negative percentage
        return float(temp[1:-2])
    if temp[-1]== '-' : # -- NA value
        return 0.0
    temp = ''.join(temp.split(',')) # remove thousand's comma
    return float(temp)


In [6]:
def get_page_soup(page):
    res = requests.get(page)
    #r = requests.get(url, headers=api_headers)
    soup = _handle_request_result_and_build_soup(res)
    return soup


In [26]:
def get_page_api(page):
    res = requests.get(website_prefix + page);
                       #, headers=api_headers, 
                       #auth=(login, password))
    if res.status_code != 200:
        print("Git error: status code", res.status_code)
    return res.json()


# Matrice de distances

In [9]:
s = get_page_soup(website_main)


In [68]:
l = s.find_all("tr")
#l


In [69]:
#l[1].find("a").text

In [67]:
#for i in range(1,51):
#   print(l[i].find("a").text)

In [29]:
g=get_page_api('Paris|Rouen')

In [34]:
g['distance']

112

In [65]:
df = pd.DataFrame();
for i in range(1,51):
    df[l[i].find("a").text]=0
for i in range(1,3):
    src = l[i].find("a").text
    row = pd.Series(name=src); #nouvelle ligne, nommee
    for j in range(1,i):
        dst = l[j].find("a").text
        row.at[dst]=df.loc[dst][src] #acces ligne
    row.at[src]=0
    for j in range(i+1,51):
        dst = l[j].find("a").text
        g=get_page_api(src + '|' + dst)
        row.at[dst]=g['distance']
    #print(row)
    df = df.append(row)
    


In [66]:
df


,Paris,Marseille,Lyon,Toulouse,Nice,Nantes,Montpellier,Strasbourg,Bordeaux,Lille,...,Saint-Paul,Nancy,Nouméa,Tourcoing,Roubaix,Nanterre,Vitry-sur-Seine,Avignon,Créteil,Dunkerque
Paris,0,661,392,589,688,343,596,397,500,204,...,6771,282,16767,216,213,11,8,578,11,243
Marseille,661,0,278,319,159,696,126,617,506,835,...,7336,604,17124,844,840,669,653,86,652,891
